In [7]:
import cv2
from PIL import Image
import numpy as np
from scipy import spatial
import os
import pytesseract
import pandas as pd
import numpy as np

In [2]:
videos = os.listdir("videos")

In [ ]:
for video in videos:
    vf = "videos/"+video
    os.mkdir("frames/"+video[:-4])
    f_path = "frames/"+video[:-4]+"/"
    # Frame Creation 
    vidcap = cv2.VideoCapture(vf)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    #print('video length ',duration,' seconds')
    #print('frames per second ',fps)
    saved_im=[]
    data_im=[]
    success,image = vidcap.read()
    filename ="frame%d.jpg" % 0;
    cv2.imwrite(f_path+filename, image)     # save frame as JPEG file  ## not saving files but writing to csv    
    success,image = vidcap.read()
    saved_im.append(filename)
    data_im.append(image)
    count = 1
    frame_list=['frame0.jpg']
    while success:
        filename ="frame%d.jpg" % count;
        count += 1
        if count % fps == 0:
            cv2.imwrite(f_path+filename, image)
            saved_im.append(filename)
            data_im.append(image)
            frame_list.append(filename)
        success,image = vidcap.read()
    df = pd.DataFrame(frame_list)
    df.to_csv('frames/'+video[:-4]+'/frame_list.csv',index=False)
    print("Completed making frames for {}".format(video))

In [ ]:
# removing Duplicate Frames
for video in videos:
    f_path = "frames/"+video[:-4]+"/"
    df = pd.read_csv(f_path+'frame_list.csv')
    frame_list=df["0"].tolist()
    comparing_frame= frame_list[0]
    final_frames= ['frame0.jpg']

    for i in range(1,len(frame_list)):
        comparing_path = f_path+comparing_frame
        frame_path = f_path+frame_list[i]
        frame= frame_list[i]
        f1 = Image.open(comparing_path)
        f2 = Image.open(frame_path)
        # i am also reducing the shape by half just to save some processing power
        f1_reshape = f1.resize((round(f1.size[0]*0.5), round(f1.size[1]*0.5)))
        f2_reshape = f2.resize((round(f1.size[0]*0.5), round(f1.size[1]*0.5)))
        # convert the images to (R,G,B) arrays
        f_array1 = np.array(f1_reshape).flatten()/255
        f_array2 = np.array(f2_reshape).flatten()/255
        similarity = -1 * (spatial.distance.cosine(f_array1, f_array2) - 1)
        if similarity<0.98:
            comparing_frame = frame_list[i]
            final_frames.append(frame)

    final_frames[:5]
    df = pd.DataFrame(final_frames)
    df.to_csv(f_path+'/final_frames.csv',index=False)
    print("Completed scanning duplicate frames for {}".format(video))

In [ ]:
# OCR 
for video in videos:
    f_path = "frames/"+video[:-4]+"/"
    df = pd.read_csv(f_path+'frame_list.csv')
    final_frames=df["0"].tolist()
    ocr_list = []
    for f in final_frames:
        path = f_path+f
        img = cv2.imread(path)
        gray = get_grayscale(img)
        
        # configurations ENG+ARABIC
        config = ('-l eng+ara --oem 0 --psm 3')
        # pytessercat
        text = pytesseract.image_to_string(gray, config=config)
        # print text
        text = text.split('\n')
        ocr_list.append(text)

    df = pd.DataFrame([final_frames,ocr_list]).T
    df.to_csv("frames/"+video[:-4]+"/ocr_list.csv",index=False)
    print("Completed OCR for {}".format(video))

In [4]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated


def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 